# Create YOLO dataset structure for training process

---

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](notebook_link)

This cookbook utilizes [sv.DetectionDataset()](https://supervision.roboflow.com/latest/datasets/core/#supervision.dataset.core.DetectionDataset) methods and other Supervision utilities to effortlessly create a YOLO dataset structure for training process

Click the `Open in Colab` button to run the cookbook on Google Colab.

In [ ]:
!pip install -q tqdm supervision==0.22.0 ultralytics

#### Import required libraries

In [ ]:
import os
import supervision as sv
from supervision.assets import download_assets, VideoAssets
from ultralytics import YOLO

#### YOLO model folders structure for training process


train\
├── images\
├── labels\
valid\
├── images\
├── labels\
test\
├── images\
├── labels\
data.yaml


<br>YOLO datasets splits in three folders `train`, `valid`, `test` and each folder must have two subfolders `images`, `labels` containing the images and their corresponding labels. Each image-label pair must have the same file name with different extension (e.g. image_001.jpg -> image_001.txt).

### 1. Download video sample to create folders with images and labels

#### ⚠️ If you have already a dataset with `images` and `labels` folders containing the images and corresponding labels, you can directly skip the following sections and go to [3. Split images and create dataset](#3-split-images-and-create-dataset).

Let's create a dataset using Supervision Assets. First, we download the video asset using ["download_assets()"](https://supervision.roboflow.com/latest/assets/#supervision.assets.downloader.download_assets) and ["VideoAssets"](https://supervision.roboflow.com/latest/assets/#supervision.assets.list.VideoAssets) and we create `images` and `labels` folders to dump video frames and extracted labels.

In [ ]:
HOME = os.getcwd()
SOURCE_VIDEO_PATH = download_assets(VideoAssets.PEOPLE_WALKING)
DATASET = f"{HOME}/dataset"
!mkdir -p {DATASET} {DATASET}/images {DATASET}/labels

As a result of executing the above `download_assets(VideoAssets.PEOPLE_WALKING)` , you will download a video file and save it at the `SOURCE_VIDEO_PATH`.

<video controls width="1280">
  <source src="https://media.roboflow.com/supervision/video-examples/people-walking.mp4" type="video/mp4">
</video>


Then using [`sv.ImageSink()`](https://supervision.roboflow.com/develop/utils/image/#supervision.utils.image.ImageSink) and [`sv.get_video_frames_generator()`](https://supervision.roboflow.com/develop/utils/video/#supervision.utils.video.get_video_frames_generator) we export all frames from the processed video.

In [ ]:
with sv.ImageSink(target_dir_path=f"{DATASET}/images", image_name_pattern="image_{:03d}.jpg") as sink:
    for image in sv.get_video_frames_generator(SOURCE_VIDEO_PATH):
        sink.save_image(image=image)

Now our images are exported in `images` folder, next step is to export the labels for each frame.\
First we load the YOLO model that we want and then we extract the results by predicting on our video.

In [ ]:
CONFIDENCE_THRESHOLD = 0.3
INFERENCE_MODEL = "yolov8n.pt"

model = YOLO("yolov8n.pt")
results = model(source="people-walking.mp4", conf=CONFIDENCE_THRESHOLD)

### 2. Create `sv.DetectionDataset()` object to define labels, images, and annotations of the dataset
1) First we extract the labels dictionary from YOLO model, and convert them to an order list that contains only the label names.<br/>

2) Using the [`sv.list_files_with_extensions()`](https://supervision.roboflow.com/develop/utils/file/#supervision.utils.file.list_files_with_extensions) and specifying the directory and files extensions, we retrieve a list of the images paths <br/>

3) We create a dictionary where keys are the image paths, and the corresponding values are the detections for each frame transformed using [`sv.Detections.from_ultralytics()`](https://supervision.roboflow.com/develop/detection/core/#supervision.detection.core.Detections.from_ultralytics)<br/>

4) We create an [`sv.DetectionDataset()`](https://supervision.roboflow.com/develop/datasets/core/#supervision.dataset.core.DetectionDataset) object by passing the `classes`, `images`, `annotations` arguments that have already been created <br/>

5) Finally, we utilize [`as_yolo()`](https://supervision.roboflow.com/develop/datasets/core/#supervision.dataset.core.DetectionDataset.as_yolo) method to export the labels to the selected path, and also create a `data.yaml` file needed for the training process

In [ ]:
classes_dict = model.model.names
classes = [class_name for class_name in classes_dict.values()]
image_paths = sv.list_files_with_extensions(directory=f"{DATASET}/images", extensions=['jpg'])

annotations_dict = {}

for image_path, detections in zip(image_paths, results):
    detections = sv.Detections.from_ultralytics(detections)
    annotations_dict[image_path] = detections

ds = sv.DetectionDataset(
    classes=classes,
    images=image_paths,
    annotations=annotations_dict
)

ds.as_yolo(
    annotations_directory_path=f"{DATASET}/labels",
    data_yaml_path=f"{DATASET}/data.yaml"
)

### 3. Split images and create dataset

Now we have an `images` folder which contains all the images of the dataset, and an `labels` folder which contains the corresponding labels of each image <br/>

Our goal is to automatically split the images and place them in `images`, `labels` subfolders based on the specified split ratios <br/>

Using [`sv.DetectionDataset.from_yolo()`](https://supervision.roboflow.com/develop/datasets/core/#supervision.dataset.core.DetectionDataset.from_yolo) we load our dataset by specifying the paths for `images`, `labels`, and `data.yaml`<br/>

We specify the `TRAIN_RATIO` and `VALID_RATIO` and then the test ratio is calculated automatically. (⚠️ Note that train and valid ratios shouldn't surpass `1.0` in total, and each value should be higher that 0 and lower than 1.0)<br/>

In order to split the dataset in individual datasets, we use [`sv.DetectionDataset.split()`](https://supervision.roboflow.com/develop/datasets/core/#supervision.dataset.core.DetectionDataset.split) method

In [ ]:
dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{DATASET}/images",
    annotations_directory_path=f"{DATASET}/labels",
    data_yaml_path=f"{DATASET}/data.yaml"
)

TRAIN_RATIO = 0.7
VALID_RATIO = 0.2

def check_ratios(train_ratio, valid_ratio):
    condition1 = (0 < train_ratio < 1.0) and (0 < valid_ratio < 1.0)
    condition2 = (train_ratio + valid_ratio) <= 1.0

    if not (condition1 and condition2):
        raise ValueError("TRAIN_RATIO and VALID_RATIO must be between 0 and 1, and their sum must not exceed 1.0")

check_ratios(TRAIN_RATIO, VALID_RATIO)

train_ds, valid_ds = dataset.split(split_ratio=TRAIN_RATIO)

_VALID_RATIO = VALID_RATIO/(1-TRAIN_RATIO)
valid_ds, test_ds = valid_ds.split(split_ratio=_VALID_RATIO)

_datasets = (train_ds, valid_ds, test_ds)
_folders = ["train", "valid", "test"]

for dataset, folder in zip(_datasets, _folders):
    dataset.as_yolo(
        images_directory_path=os.path.join(f"{DATASET}", folder, "images"),
        annotations_directory_path=os.path.join(f"{DATASET}", folder, "labels"),
        data_yaml_path=f"{DATASET}/data.yaml"
    )

### 4. Model training

Now `train`, `valid`, `test` folders and `data.yaml` are ready for training process!

In [ ]:
model.train(data = f"{DATASET}/data.yaml", epochs = 10, imgsz = 640, batch = 32, lr0 = 0.001)